# Diving into Delta Lake: Unpacking the Transaction Log



### Steps to run this notebook

You can run this notebook in a Databricks environment. Specifically, this notebook has been designed to run in [Databricks Community Edition](http://community.cloud.databricks.com/) as well.
To run this notebook, you have to [create a cluster](https://docs.databricks.com/clusters/create.html) with version **Databricks Runtime 6.1 or later** and [attach this notebook](https://docs.databricks.com/notebooks/notebooks-manage.html#attach-a-notebook-to-a-cluster) to that cluster. <br/>&nbsp;

### Source Data for this notebook

The data used is a modified version of the public data from [Lending Club](https://www.kaggle.com/wendykan/lending-club-loan-data). It includes all funded loans from 2012 to 2017. Each loan includes applicant information provided by the applicant as well as the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. For a full view of the data please view the data dictionary available [here](https://resources.lendingclub.com/LCDataDictionary.xlsx).

<img src="https://docs.delta.io/latest/_static/delta-lake-logo.png" width=300/>

An open-source storage format that brings ACID transactions to Apache Spark™ and big data workloads.

* **Open format**: Stored as Parquet format in blob storage.
* **ACID Transactions**: Ensures data integrity and read consistency with complex, concurrent data pipelines.
* **Schema Enforcement and Evolution**: Ensures data cleanliness by blocking writes with unexpected.
* **Audit History**: History of all the operations that happened in the table.
* **Time Travel**: Query previous versions of the table by time or version number.
* **Deletes and upserts**: Supports deleting and upserting into tables with programmatic APIs.
* **Scalable Metadata management**: Able to handle millions of files are scaling the metadata operations with Spark.
* **Unified Batch and Streaming Source and Sink**: A table in Delta Lake is both a batch table, as well as a streaming source and sink. Streaming data ingest, batch historic backfill, and interactive queries all just work out of the box.

## Explore data as a Parquet table

#####Download the sampled Lending Club data

In [0]:
%sh mkdir -p /dbfs/tmp/sais_eu_19_demo/loans/ && wget -O /dbfs/tmp/sais_eu_19_demo/loans/SAISEU19-loan-risks.snappy.parquet  https://pages.databricks.com/rs/094-YMS-629/images/SAISEU19-loan-risks.snappy.parquet && ls -al  /dbfs/tmp/sais_eu_19_demo/loans/ 

--2021-06-01 04:16:40-- https://pages.databricks.com/rs/094-YMS-629/images/SAISEU19-loan-risks.snappy.parquet
Resolving pages.databricks.com (pages.databricks.com)... 104.17.72.206, 104.17.73.206, 104.17.74.206, ...
Connecting to pages.databricks.com (pages.databricks.com)|104.17.72.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 164631 (161K) [text/plain]
Saving to: ‘/dbfs/tmp/sais_eu_19_demo/loans/SAISEU19-loan-risks.snappy.parquet’

 0K .......... .......... .......... .......... .......... 31% 5.63M 0s
 50K .......... .......... .......... .......... .......... 62% 6.31M 0s
 100K .......... .......... .......... .......... .......... 93% 6.96M 0s
 150K .......... 100% 4.64M=0.03s

2021-06-01 04:16:40 (6.11 MB/s) - ‘/dbfs/tmp/sais_eu_19_demo/loans/SAISEU19-loan-risks.snappy.parquet’ saved [164631/164631]

total 172
drwxr-xr-x 2 root root 4096 Jun 1 04:12 .
drwxr-xr-x 3 root root 4096 Jun 1 04:12 ..
-rw-r--r-- 1 root root 164631 May 11 13:00 SAISEU19-loan-risks.snappy.parquet

**Setup and configuration**

In [0]:
import os, shutil

# Configurations necessary for running of Databricks Community Edition
spark.sql("set spark.sql.shuffle.partitions = 1")
spark.sql("set spark.databricks.delta.snapshotPartitions = 1")

demo_path = "/sais_eu_19_demo/"

if os.path.exists("/dbfs" + demo_path):
  print("Deleting path " + demo_path)
  shutil.rmtree("/dbfs" + demo_path)
  print("Deleted " + demo_path)

#####Create the parquet table "loans_parquet"

In [0]:
%sh cp    /dbfs/tmp/sais_eu_19_demo/loans/SAISEU19-loan-risks.snappy.parquet /dbfs/tmp/sais_eu_19_demo/loans_new/

cp: cannot create regular file '/dbfs/tmp/sais_eu_19_demo/loans_new/': Not a directory

In [0]:
import os
import shutil
from pyspark.sql.functions import * 

parquet_path = "/sais_eu_19_demo/loans_parquet"

# Delete a new parquet table with the parquet file
if os.path.exists("/dbfs" + parquet_path):
  print("Deleting path " + parquet_path)
  shutil.rmtree("/dbfs" + parquet_path)
  
# Create a new parquet table with the parquet file
spark.read.format("parquet").load("/tmp/sais_eu_19_demo/loans") \
  .write.format("parquet").save(parquet_path)
print("Created a Parquet table at " + parquet_path)

# Create a view on the table called loans_parquet
spark.read.format("parquet").load(parquet_path).createOrReplaceTempView("loans_parquet")
print("Defined view 'loans_parquet'")


--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3637542605724024> in <module> 
 11 
 12 # Create a new parquet table with the parquet file 
 ---> 13 spark . read . format ( "parquet" ) . load ( "/tmp/sais_eu_19_demo/loans" ) \ 
 14 . write . format ( "parquet" ) . save ( parquet_path ) 
 15 print ( "Created a Parquet table at " + parquet_path ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 202 self . options ( ** options ) 
 203 if isinstance ( path , str ) : 
 --> 204 return self . _df ( self . _jreader . load ( path ) ) 
 205 elif path is not None : 
 206 if type ( path ) != list : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 114 # Hide where the exception came from that shows a non-Pythonic 
 115 # JVM exception message. 
 --> 116 raise converted from None 
 117 else : 
 118 raise 

 AnalysisException : Unable to infer schema for Parquet. It must be specified manually.

#####Let's explore this parquet table.

*Schema of the table*
- load_id - unique id for each loan
- funded_amnt - principal amount of the loan funded to the loanee
- paid_amnt - amount from the principle that has been paid back (ignoring interests)
- addr_state - state where this loan was funded

In [0]:
spark.sql("select * from loans_parquet").show(20)

**How many records does it have?**

In [0]:
spark.sql("select count(*) from loans_parquet").show()

In [0]:
dbutils.notebook.exit("stop") # Stop the notebook before the streaming cell, in case of a "run all" 

**Let's start appending some new data to it using Structured Streaming.**

We will generate a stream of data from with randomly generated loan ids and amounts. 
In addition, we are going to define a few more useful utility functions.

In [0]:
import random
import os
from pyspark.sql.functions import *
from pyspark.sql.types import *


def random_checkpoint_dir(): 
  return "/sais_eu_19_demo/chkpt/%s" % str(random.randint(0, 10000))

# User-defined function to generate random state

states = ["CA", "TX", "NY", "WA"]

@udf(returnType=StringType())
def random_state():
  return str(random.choice(states))

# Function to start a streaming query with a stream of randomly generated load data and append to the parquet table
def generate_and_append_data_stream(table_format, table_path):
  
  stream_data = spark.readStream.format("rate").option("rowsPerSecond", 5).load() \
    .withColumn("loan_id", 10000 + col("value")) \
    .withColumn("funded_amnt", (rand() * 5000 + 5000).cast("integer")) \
    .withColumn("paid_amnt", col("funded_amnt") - (rand() * 2000)) \
    .withColumn("addr_state", random_state()) \

  query = stream_data.writeStream \
    .format(table_format) \
    .option("checkpointLocation", random_checkpoint_dir()) \
    .trigger(processingTime = "10 seconds") \
    .start(table_path)

  return query

# Function to stop all streaming queries 
def stop_all_streams():
  # Stop all the streams
  print("Stopping all streams")
  for s in spark.streams.active:
    s.stop()
  print("Stopped all streams")
  print("Deleting checkpoints")  
  dbutils.fs.rm("/sais_eu_19_demo/chkpt/", True)
  print("Deleted checkpoints")

**Let's start a new stream to append data to the Parquet table**

In [0]:
stream_query = generate_and_append_data_stream(
    table_format = "parquet", 
    table_path = parquet_path)

**Let's see if the data is being added to the table or not**.

In [0]:
spark.read.format("parquet").load(parquet_path).count()

** What happens when we try to add a second stream?**

In [0]:
stream_query2 = generate_and_append_data_stream(
    table_format = "parquet", 
    table_path = parquet_path)

**Where did our existing 14705 rows go? Let's see the data once again**

In [0]:
spark.read.format("parquet").load(parquet_path).show() # wrong schema!

**Where did the two new columns `timestamp` and `value` come from? What happened here!**

What really happened is that when the streaming query started adding new data to the Parquet table, it did not properly account for the existing data in the table. Furthermore, the new data files that written out accidentally had two extra columns in the schema. Hence, when reading the table, the 2 different schema from different files were merged together, thus unexpectedly modifying the schema of the table.


Before we move on, **if you are running on Databricks Community Edition, definitely stop the streaming queries.** 

You free account in Databricks Community Edition has quota limits on the number of files and we do not want to hit that quote limit by running the streaming queries for too long.

In [0]:
stop_all_streams()

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Batch + stream processing and schema enforcement with Delta Lake
Let's understand Delta Lake solves these particular problems (among many others). We will start by creating a Delta table from the original data.

In [0]:
# Configure Delta Lake Silver Path
delta_path = "/sais_eu_19_demo/loans_delta"

# Configurations necessary for running of Databricks Community Edition
spark.sql("set spark.sql.shuffle.partitions = 1")
spark.sql("set spark.databricks.delta.snapshotPartitions = 1")

# Remove folder if it exists
print("Deleting directory " + delta_path)
dbutils.fs.rm(delta_path, recurse=True)

# Create the Delta table with the same loans data
spark.read.format("parquet").load("/tmp/sais_eu_19_demo/loans/SAISEU19-loan-risks.snappy.parquet") \
  .write.format("delta").save(delta_path)
print("Created a Delta table at " + delta_path)

spark.read.format("delta").load(delta_path).createOrReplaceTempView("loans_delta")
print("Defined view 'loans_delta'")

**Let's see the data once again.**

In [0]:
spark.sql("select count(*) from loans_delta").show()

In [0]:
spark.sql("select * from loans_delta").show()

###  ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Schema Enforcement

**Let's run a streaming count(*) on the table so that the count updates automatically**

In [0]:
spark.readStream.format("delta").load(delta_path).createOrReplaceTempView("loans_delta_stream")
display(spark.sql("select count(*) from loans_delta_stream"))

**Now let's try writing the streaming appends once again**

In [0]:
stream_query_2 = generate_and_append_data_stream(table_format = "delta", table_path = delta_path)

The writes were blocked because the schema of the new data did not match the schema of table (see the exception details). See more information about how it works [here](https://databricks.com/blog/2019/09/24/diving-into-delta-lake-schema-enforcement-evolution.html).

**Now, let's fix the streaming query by selecting the columns we want to write.**

In [0]:
from pyspark.sql.functions import *

# Generate a stream of randomly generated load data and append to the parquet table
def generate_and_append_data_stream_fixed(table_format, table_path):
    
  stream_data = spark.readStream.format("rate").option("rowsPerSecond", 50).load() \
    .withColumn("loan_id", 10000 + col("value")) \
    .withColumn("funded_amnt", (rand() * 5000 + 5000).cast("integer")) \
    .withColumn("paid_amnt", col("funded_amnt") - (rand() * 2000)) \
    .withColumn("addr_state", random_state()) \
    .select("loan_id", "funded_amnt", "paid_amnt", "addr_state")   # *********** FIXED THE SCHEMA OF THE GENERATED DATA *************

  query = stream_data.writeStream \
    .format(table_format) \
    .option("checkpointLocation", random_checkpoint_dir()) \
    .trigger(processingTime="10 seconds") \
    .start(table_path)

  return query


**Now we can successfully write to the table. Note the count in the above streaming query increasing as we write to this table.**

In [0]:
stream_query_2 = generate_and_append_data_stream_fixed(table_format = "delta", table_path = delta_path)

**Scroll back up to see the numbers change in the `readStream` as more data is being appended by the `writeStream`.** 

**In fact, we can run multiple concurrent streams writing to that table, it will work together.**

In [0]:
stream_query_3 = generate_and_append_data_stream_fixed(table_format = "delta", table_path = delta_path)

Just for sanity check, let's query as a batch

Note, you can run a read stream, two write streams, and read in batch - concurrently!

In [0]:
display(spark.sql("select count(*) from loans_delta"))

**Again, remember to stop all the streaming queries.**

In [0]:
stop_all_streams()

### ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Audit Delta Lake Table History
All changes to the Delta table are recorded as commits in the table's transaction log. As you write into a Delta table or directory, every operation is automatically versioned. You can use the HISTORY command to view the table's history. For more information, check out the [docs](https://docs.delta.io/latest/delta-utility.html#history).

In [0]:
%sh 
ls -lt /dbfs/sais_eu_19_demo/loans_delta/_delta_log/

In [0]:
from delta.tables import *
delta_path = "/sais_eu_19_demo/loans_delta"

deltaTable = DeltaTable.forPath(spark, delta_path)
display(deltaTable.history())

In [0]:
%sh 
head /dbfs/sais_eu_19_demo/loans_delta/_delta_log/00000000000000000011.json

** With Time Travel, you can query previous versions **

In [0]:
currentVersion = deltaTable.history(1).select("version").collect()[0][0]

v01 = spark.read.format("delta").option("versionAsOf", 1).load(delta_path).count()
v11 = spark.read.format("delta").option("versionAsOf", 11).load(delta_path).count()
vno = spark.read.format("delta").option("versionAsOf", currentVersion).load(delta_path).count()
print("loans_delta table counts:\n Initial [%s] \n Version 11 [%s] \n Current Version [%s]" % (v01, v11, vno))

** But, there are a lot of files because of the data versioning.**

In [0]:
%sh 
ls -lt /dbfs/sais_eu_19_demo/loans_delta/

%md ###  ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Compact files

If you continuously write data to a Delta table, it will over time accumulate a large number of files, especially if you add data in small batches. This can have an adverse effect on the efficiency of table reads, and it can also affect the performance of your file system. Ideally, a large number of small files should be rewritten into a smaller number of larger files on a regular basis. This is known as compaction.

You can compact a table by repartitioning it to smaller number of files. In addition, you can specify the option `dataChange` to be false indicates that the operation does not change the data, only rearranges the data layout. This would ensure that other concurrent operations are minimally affected due to this compaction operation.

In [0]:
numFiles = 4

(spark.read
   .format("delta")
   .load(delta_path)
   .repartition(numFiles)
   .write
   .option("dataChange", "false")
   .format("delta")
   .mode("overwrite")
   .save(delta_path))

In [0]:
%sh 
ls -lt /dbfs/sais_eu_19_demo/loans_delta/ | wc -l

In [0]:
# This is an anti-pattern
spark.sql("SET spark.databricks.delta.retentionDurationCheck.enabled = false")
deltaTable.vacuum(retentionHours = 0)

In [0]:
%sh 
ls -lt /dbfs/sais_eu_19_demo/loans_delta/

#Join the community!


* [Delta Lake on GitHub](https://github.com/delta-io/delta)
* [Delta Lake Slack Channel](https://delta-users.slack.com/) ([Registration Link](https://join.slack.com/t/delta-users/shared_invite/enQtNTY1NDg0ODcxOTI1LWJkZGU3ZmQ3MjkzNmY2ZDM0NjNlYjE4MWIzYjg2OWM1OTBmMWIxZTllMjg3ZmJkNjIwZmE1ZTZkMmQ0OTk5ZjA))
* [Public Mailing List](https://groups.google.com/forum/#!forum/delta-users)